In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# SECTION 1: SIMULATE THE DATABASE TABLES

print("Step 1: Simulating database tables...")

# 'users' table: Contains user information and acquisition channel
users_data = {
    'user_id': range(1, 101),
    'sign_up_date': pd.to_datetime(pd.date_range(start='2024-01-01', periods=100, freq='D')),
    'acquisition_channel': np.random.choice(['Google Search', 'Facebook', 'Instagram', 'Organic'], 100)
}
users = pd.DataFrame(users_data)

# 'app_events' table: Contains all user actions
events_data = {
    'user_id': np.random.choice(range(1, 101), 500, replace=True),
    'timestamp': pd.to_datetime(pd.date_range(start='2024-01-01', periods=500, freq='H')),
    'event_name': np.random.choice(['story_read', 'chapter_purchase', 'subscription_start', 'app_open', 'story_browse'], 500)
}
app_events = pd.DataFrame(events_data)
for i in range(1, 10):
    app_events = pd.concat([app_events, pd.DataFrame({'user_id': [i, i], 'timestamp': [datetime.now() - timedelta(days=np.random.randint(1, 10)), datetime.now() - timedelta(days=np.random.randint(1, 10))], 'event_name': ['chapter_purchase', 'subscription_start']})])
app_events['timestamp'] = pd.to_datetime(app_events['timestamp'])

# 'campaigns' table: Contains campaign costs
# We need to explicitly define this table to merge the costs correctly later.
campaigns_data = {
    'campaign_name': ['Google Search', 'Facebook', 'Instagram', 'Organic'],
    'cost': [500, 750, 300, 0]
}
campaigns = pd.DataFrame(campaigns_data)

# SECTION 2: SIMULATE EXECUTION OF YOUR SQL QUERIES

print("\nStep 2: Simulating Deliverable 1 (User Acquisition & Conversion) query...")
key_actions = app_events[app_events['event_name'].isin(['story_read', 'chapter_purchase'])].groupby('user_id')['timestamp'].min().reset_index()
key_actions = key_actions.rename(columns={'timestamp': 'first_key_action_date'})
df_acq_conv = users.merge(key_actions, on='user_id', how='left')
df_acq_conv['has_converted'] = df_acq_conv['first_key_action_date'].notna().astype(int)

print("Simulating Deliverable 2 (Campaign Performance & ROI) query...")
app_events['revenue'] = app_events['event_name'].map({'chapter_purchase': 1.99, 'subscription_start': 9.99}).fillna(0)
total_revenue_per_user = app_events.groupby('user_id')['revenue'].sum().reset_index()
total_revenue_per_user.rename(columns={'revenue': 'total_revenue'}, inplace=True)

print("Simulating Deliverable 3 (Audience Segmentation) query...")
most_recent_date = app_events['timestamp'].max()
user_rfm = app_events.groupby('user_id').agg(
    recency_days=('timestamp', lambda x: (most_recent_date - x.max()).days),
    frequency=('event_name', lambda x: x.isin(['chapter_purchase', 'subscription_start']).sum()),
    monetary=('revenue', 'sum')
).reset_index()

user_rfm['r_score'] = pd.qcut(user_rfm['recency_days'], 4, labels=[4, 3, 2, 1], duplicates='drop')
user_rfm['f_score'] = pd.qcut(user_rfm['frequency'], 4, labels=[1, 2, 3, 4], duplicates='drop')
user_rfm['m_score'] = pd.qcut(user_rfm['monetary'], 4, labels=[1, 2, 3, 4], duplicates='drop')
user_rfm.fillna(1, inplace=True)

def assign_segment(row):
    r, f, m = row['r_score'], row['f_score'], row['m_score']
    if r == 4 and f == 4 and m == 4: return 'Champions'
    elif r in [4, 3] and f == 4 and m == 4: return 'Loyal Champions'
    elif r == 4 and f == 3 and m == 3: return 'Loyal'
    elif r in [4, 3] and f in [2, 3] and m in [3, 4]: return 'Potential Loyalists'
    elif r in [2, 3] and f == 4 and m in [3, 4]: return 'Loyal'
    elif r in [3, 4] and f in [1, 2] and m in [1, 4]: return 'New Customers'
    elif r in [1, 2] and f in [3, 4] and m in [3, 4]: return 'At Risk'
    elif r in [1, 2] and f in [1, 2] and m in [1, 2]: return 'Hibernating'
    else: return 'Other'

user_rfm['customer_segment'] = user_rfm.apply(assign_segment, axis=1)
user_rfm['r_score'] = user_rfm['r_score'].astype(int)
user_rfm['f_score'] = user_rfm['f_score'].astype(int)
user_rfm['m_score'] = user_rfm['m_score'].astype(int)

# SECTION 3: JOIN ALL RESULTS INTO A SINGLE DATAFRAME FOR TABLEAU

print("\nStep 3: Joining all results into a single DataFrame...")
# Start with the main 'users' table
df_final = users.copy()

# Join with RFM results
df_final = pd.merge(df_final, user_rfm, on='user_id', how='left')

# Join with acquisition and conversion data
df_final = pd.merge(df_final, df_acq_conv[['user_id', 'has_converted']], on='user_id', how='left')

# Join with total revenue per user
df_final = pd.merge(df_final, total_revenue_per_user, on='user_id', how='left')
df_final.rename(columns={'total_revenue': 'total_lifetime_revenue'}, inplace=True)

# CRITICAL FIX: Merge the campaign costs into the final DataFrame
df_final = pd.merge(df_final, campaigns, left_on='acquisition_channel', right_on='campaign_name', how='left')
df_final.drop(columns=['campaign_name'], inplace=True) # Clean up the extra column

# Final cleanup and fill NaNs
df_final['has_converted'].fillna(0, inplace=True)
df_final['total_lifetime_revenue'].fillna(0, inplace=True)
df_final['cost'].fillna(0, inplace=True) # Fill cost for organic channel
df_final['recency_days'].fillna(365, inplace=True)
df_final['frequency'].fillna(0, inplace=True)
df_final['monetary'].fillna(0, inplace=True)
df_final['r_score'].fillna(1, inplace=True)
df_final['f_score'].fillna(1, inplace=True)
df_final['m_score'].fillna(1, inplace=True)
df_final['customer_segment'].fillna('Hibernating', inplace=True)
# SECTION 4: EXPORT THE FINAL DATAFRAME TO A CSV FILE

output_file = 'marketing_analytics_dashboard_data_v2.csv'
df_final.to_csv(output_file, index=False)

print(f"\nSuccess! All data has been combined and exported to '{output_file}'.")
print("\nYou can now import this file into Tableau to build your dashboard.")
print("\nFinal DataFrame head:")
print(df_final.head())

Step 1: Simulating database tables...


C:\Users\AKE\AppData\Local\Temp\ipykernel_8820\1691144041.py:21: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  'timestamp': pd.to_datetime(pd.date_range(start='2024-01-01', periods=500, freq='H')),



Step 2: Simulating Deliverable 1 (User Acquisition & Conversion) query...
Simulating Deliverable 2 (Campaign Performance & ROI) query...
Simulating Deliverable 3 (Audience Segmentation) query...

Step 3: Joining all results into a single DataFrame...

Success! All data has been combined and exported to 'marketing_analytics_dashboard_data_v2.csv'.

You can now import this file into Tableau to build your dashboard.

Final DataFrame head:
   user_id sign_up_date acquisition_channel  recency_days  frequency  \
0        1   2024-01-01            Facebook             3          6   
1        2   2024-01-02       Google Search             0          6   
2        3   2024-01-03            Facebook             3          2   
3        4   2024-01-04            Facebook             6          3   
4        5   2024-01-05            Facebook             2          3   

   monetary  r_score  f_score  m_score customer_segment  has_converted  \
0     35.94        4        4        4        Champi

C:\Users\AKE\AppData\Local\Temp\ipykernel_8820\1691144041.py:103: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_final['has_converted'].fillna(0, inplace=True)
C:\Users\AKE\AppData\Local\Temp\ipykernel_8820\1691144041.py:104: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exam